In [16]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [['word', '단어'], ['wood', '나무'], 
            ['game', '놀이'], ['girl', '소녀'], 
            ['kiss', '키스'], ['love', '사랑']]


def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)

    return input_batch, output_batch, target_batch

learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess=tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
print('최적화 완료!')

def translate(word):
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated
print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('lebu ->', translate('lebu'))

Epoch: 0001 cost = 3.747611
Epoch: 0002 cost = 2.688047
Epoch: 0003 cost = 1.829976
Epoch: 0004 cost = 1.283864
Epoch: 0005 cost = 0.679634
Epoch: 0006 cost = 0.468983
Epoch: 0007 cost = 0.345612
Epoch: 0008 cost = 0.226724
Epoch: 0009 cost = 0.261870
Epoch: 0010 cost = 0.198833
Epoch: 0011 cost = 0.138407
Epoch: 0012 cost = 0.126832
Epoch: 0013 cost = 0.057473
Epoch: 0014 cost = 0.071633
Epoch: 0015 cost = 0.084347
Epoch: 0016 cost = 0.075183
Epoch: 0017 cost = 0.073959
Epoch: 0018 cost = 0.043591
Epoch: 0019 cost = 0.035085
Epoch: 0020 cost = 0.042100
Epoch: 0021 cost = 0.034530
Epoch: 0022 cost = 0.005579
Epoch: 0023 cost = 0.004919
Epoch: 0024 cost = 0.008079
Epoch: 0025 cost = 0.014102
Epoch: 0026 cost = 0.001565
Epoch: 0027 cost = 0.002849
Epoch: 0028 cost = 0.002801
Epoch: 0029 cost = 0.006049
Epoch: 0030 cost = 0.007084
Epoch: 0031 cost = 0.004271
Epoch: 0032 cost = 0.004406
Epoch: 0033 cost = 0.002267
Epoch: 0034 cost = 0.003570
Epoch: 0035 cost = 0.004211
Epoch: 0036 cost = 0